# Transformadas espaciales con Robotics Toolbox (Python)
Este cuaderno resume rotaciones SO(3), transformaciones homogéneas SE(3), composición, inversa y aplicación a puntos utilizando `spatialmath` y `roboticstoolbox`.

In [ ]:
import numpy as np
import spatialmath as sm
import roboticstoolbox as rtb
print('spatialmath', sm.__version__)
print('roboticstoolbox', rtb.__version__)

## 1. Rotaciones en SO(3)
Creamos rotaciones elementales alrededor de **x**, **y**, **z** y verificamos propiedades.

In [ ]:
Rx = sm.SO3.Rx(np.deg2rad(30))  # 30° sobre x
Ry = sm.SO3.Ry(np.deg2rad(45))  # 45° sobre y
Rz = sm.SO3.Rz(np.deg2rad(60))  # 60° sobre z
R = Rz * Ry * Rx  # composición (aplica a derecha)
print('R\n', R.A)
print('Determinante:', np.linalg.det(R.A))

## 2. Transformaciones homogéneas en SE(3)
Agregamos traslaciones y componemos marcos.

In [ ]:
T1 = sm.SE3(0.5, 0.0, 0.2) * sm.SE3.Rz(np.deg2rad(45))
T2 = sm.SE3(0.0, 0.3, 0.0) * sm.SE3.Rx(np.deg2rad(-30))
TAB = T1  # A->B
TBC = T2  # B->C
TAC = TAB * TBC  # A->C
print('TAB =\n', TAB.A)
print('TBC =\n', TBC.A)
print('TAC = TAB*TBC =\n', TAC.A)

## 3. Aplicación de transformaciones a puntos
Aplicamos `T` a un conjunto de puntos y verificamos la inversa.

In [ ]:
P = np.array([[0.1, 0.0, 0.0],
              [0.0, 0.1, 0.0],
              [0.0, 0.0, 0.1]]).T  # 3x3, columnas= puntos
Q = TAC * P  # aplica a cada columna
Pinv = TAC.inv() * Q
print('P original =\n', P)
print('Q = T*P =\n', Q)
print('T^{-1}*Q (debe ~P) =\n', Pinv)

## 4. RPY / Euler y equivalencias
Convertimos entre representaciones.

In [ ]:
R = sm.SO3.RPY([10, 20, 30], unit='deg', order='zyx')
rpy = R.rpy(order='zyx', unit='deg')
angax = R.angvec()
print('R (RPY zyx=10,20,30) =\n', R.A)
print('RPY recuperado (deg) =', rpy)
print('Ángulo-eje =', angax)

## 5. Demostración con un manipulador de RTB
Obtenemos un modelo y mostramos su pose de herramienta tras aplicar una configuración.

In [ ]:
puma = rtb.models.DH.Puma560()
q = np.deg2rad([0, -45, 30, 0, 45, 0])
T = puma.fkine(q)
print(puma)
print('Tee =\n', T.A)

## 6. (Opcional) Visualización rápida
Si ejecutas en un entorno con interfaz gráfica, puedes visualizar con Swift:
```python
env = rtb.backends.Swift()
env.launch()
env.add(puma)
puma.q = q
```
_Cierra la ventana para continuar._